In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report
from imutils import paths
import argparse
import pickle
import cv2

In [2]:
#from google.colab import drive

#drive.mount('/content/drive')

In [3]:
#### Load the dataset from CSV file
df = pd.read_csv('FungiCLEF2023_train_metadata_PRODUCTION.csv')

#### Find the top 100 species by number of examples
top_100_species = df['species'].value_counts().head(100).index.tolist()

#### Filter the dataframe to only include rows from the top 100 species
df_top_100 = df[df['species'].isin(top_100_species)]

#### Create a new dataframe with only the 'image_path' and 'class_id' columns
top_100_pairs = df_top_100[['image_path', 'species']]

#### Print the shape of the final dataframe
print(f"Final dataframe shape: {top_100_pairs.shape}")

unique_species_ids = top_100_pairs['species'].nunique()
print(f"Number of unique species: {unique_species_ids}")
print(top_100_species)

# Create a dictionary that maps each species to a unique integer label
species_labels = {species: index for index, species in enumerate(top_100_species)}

print(species_labels)

Final dataframe shape: (84969, 2)
Number of unique species: 100
['Trametes versicolor', 'Fomitopsis pinicola', 'Hypholoma fasciculare', 'Pluteus cervinus', 'Fomes fomentarius', 'Bjerkandera adusta', 'Auricularia auricula-judae', 'Coprinellus micaceus', 'Stereum hirsutum', 'Mycena galericulata', 'Plicaturopsis crispa', 'Cuphophyllus virgineus', 'Hygrocybe miniata', 'Clitocybe nebularis', 'Ganoderma applanatum', 'Daedaleopsis confragosa', 'Pleurotus ostreatus', 'Tremella mesenterica', 'Gymnopilus penetrans', 'Meripilus giganteus', 'Armillaria lutea', 'Amanita muscaria', 'Tubaria furfuracea', 'Leccinum scabrum', 'Neoboletus luridiformis', 'Imleria badia', 'Lycoperdon perlatum', 'Phaeolepiota aurea', 'Trametes hirsuta', 'Trametes gibbosa', 'Boletus edulis', 'Pseudocraterellus undulatus', 'Byssomerulius corium', 'Schizophyllum commune', 'Kuehneromyces mutabilis', 'Xylaria hypoxylon', 'Daedalea quercina', 'Laccaria amethystina', 'Xerocomellus chrysenteron', 'Coprinus comatus', 'Phlebia treme

In [4]:
data_dir = 'top_100_species_images/top_100_species_images'

num_classes = 100
num_epochs = 20
k_folds = 5
batch_size = 32
image_size = (224, 224)

In [5]:
# Prepare data
def load_data(data_dir):
    images = []
    labels = []
    image_count = 0
    label_count = 0
    iterc = 0

    for file_name in os.listdir(data_dir):
        if file_name.endswith('.JPG'):
            if (iterc % 2) == 0:
                # Load the image
                image_path = os.path.join(data_dir, file_name)
                img = cv2.imread(image_path)

                # Resize image while preserving aspect ratio
                height, width, _ = img.shape
                if height > width:
                    new_height = 224
                    new_width = int(width * (224 / height))
                else:
                    new_width = 224
                    new_height = int(height * (224 / width))
                img = cv2.resize(img, (new_width, new_height))

                # Pad the shorter side with black pixels
                delta_w = 224 - new_width
                delta_h = 224 - new_height
                top, bottom = delta_h // 2, delta_h - (delta_h // 2)
                left, right = delta_w // 2, delta_w - (delta_w // 2)
                img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])

                images.append(img)  # Store the image data
                image_count += 1

                # Extract the label from the file name
                image_path_after_slash = os.path.basename(file_name)
                species_name = top_100_pairs.loc[top_100_pairs['image_path'] == image_path_after_slash, 'species'].values[0]

                # Ensure the species name is in the mapping
                if species_name in species_labels:
                    label = species_labels[species_name]
                    labels.append(label)  # Store the corresponding label
                    label_count += 1
                else:
                    print(f"Warning: {species_name} not found in species_labels mapping.")

            iterc += 1

            if iterc % 100 == 0:
                print(iterc)

    print(len(set(labels)))
    print("Root directory:", data_dir)  # Print the root directory
    print("Image count:", image_count)
    print("Label count:", label_count)

    return np.array(images), np.array(labels)

In [6]:
# Load images and labels
X, y = load_data(data_dir)
y = to_categorical(y, num_classes)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [7]:
import csv
from sklearn.metrics import f1_score
from keras import backend as K
import gc
import tensorflow as tf

# Create a list to store the metrics for each fold
fold_metrics = []

# K-Fold Cross Validation
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

for fold, (train_index, val_index) in enumerate(kf.split(X)):
    print(f'Fold {fold + 1}/{k_folds}')

    if fold != 0:
        continue

    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Convert data to TensorFlow datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))

    # Data augmentation and preparation
    train_dataset = train_dataset.batch(batch_size).repeat()  # Repeats indefinitely for training
    val_dataset = val_dataset.batch(batch_size)  # No repeat for validation

    # Load the ResNet50 model
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    headModel = base_model.output
    headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(256, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(256, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(num_classes, activation="softmax")(headModel)

    model = Model(inputs=base_model.input, outputs=headModel)

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Compile the model
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        steps_per_epoch=len(X_train) // batch_size,
        validation_steps=len(X_val) // batch_size,
        epochs=num_epochs
    )

    # Calculate F1 score on validation data
    val_preds = model.predict(val_dataset)
    val_preds_classes = np.argmax(val_preds, axis=1)
    val_true_classes = np.argmax(y_val, axis=1)
    val_f1_score = f1_score(val_true_classes, val_preds_classes, average='weighted')

    # Save the metrics for the current fold
    fold_metrics.append({
        'fold': fold + 1,
        'train_accuracy': history.history['accuracy'][-1],
        'train_loss': history.history['loss'][-1],
        'val_accuracy': history.history['val_accuracy'][-1],
        'val_loss': history.history['val_loss'][-1],
        'val_f1_score': val_f1_score
    })

    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'Model accuracy - Fold {fold + 1}')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    # Clear memory
    K.clear_session()
    gc.collect()

# Save the final model
model.save('resnet50_kfold_classifier.h5')

Fold 1/5
Epoch 1/20
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 2004s 2s/step - accuracy: 0.0522 - loss: 4.3517 - val_accuracy: 0.1196 - val_loss: 3.8758
Epoch 2/20
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 1606s 2s/step - accuracy: 0.0964 - loss: 3.8818 - val_accuracy: 0.1765 - val_loss: 3.3162
Epoch 3/20


c:\Users\lenovo\.pyenv\pyenv-win\versions\3.10.11\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1062/1062 ━━━━━━━━━━━━━━━━━━━━ 2005s 2s/step - accuracy: 0.1228 - loss: 3.6624 - val_accuracy: 0.1868 - val_loss: 3.3535
Epoch 4/20
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 1603s 2s/step - accuracy: 0.1412 - loss: 3.4962 - val_accuracy: 0.3529 - val_loss: 2.9081
Epoch 5/20
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 2007s 2s/step - accuracy: 0.1527 - loss: 3.4085 - val_accuracy: 0.2250 - val_loss: 3.1567
Epoch 6/20
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 1608s 2s/step - accuracy: 0.1627 - loss: 3.3435 - val_accuracy: 0.2941 - val_loss: 2.7274
Epoch 7/20
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 2021s 2s/step - accuracy: 0.1689 - loss: 3.2869 - val_accuracy: 0.2328 - val_loss: 3.0707
Epoch 8/20
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 1630s 2s/step - accuracy: 0.1741 - loss: 3.2675 - val_accuracy: 0.2941 - val_loss: 2.6641
Epoch 9/20
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 2035s 2s/step - accuracy: 0.1831 - loss: 3.2056 - val_accuracy: 0.2487 - val_loss: 2.9969
Epoch 10/20
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 1610s 2s/step - accuracy: 0.1900 - loss: 3.1

KeyboardInterrupt: 

In [8]:
import csv

# Save the metrics to a CSV file
with open('training_metrics.csv', 'w', newline='') as file:
    fieldnames = ['fold', 'train_accuracy', 'train_loss', 'val_accuracy', 'val_loss', 'val_f1_score']
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()
    for metrics in fold_metrics:
        writer.writerow(metrics)